# About: Moodle環境の削除--EC2インスタンスの削除

---

EC2インスタンスを削除します。

## パラメータの設定

EC2インスタンスを削除するのに必要となるパラメータを設定します。

### AWSの認証情報

EC2インスタンスの操作を行うためのAWSのアクセスキーとシークレットキーを入力します。

アクセスキーの値を入力してください。

In [ ]:
from getpass import getpass
aws_access_key = getpass()

シークレットキーを入力してください。

In [ ]:
aws_secret_key = getpass()

このNotebookで実行するコマンドから、AWSの認証情報を参照できるようにするため入力された情報を環境変数に設定します。

In [ ]:
import os

os.environ['AWS_ACCESS_KEY_ID'] = aws_access_key
os.environ['AWS_SECRET_ACCESS_KEY'] = aws_secret_key

### 削除対象となるインスタンスIDの指定

削除対象となるEC2インスタンスのIDを指定します。

削除候補となるインスタンスのIDリストを表示するために、Ansibleの `group_vars` ファイルを読み込みます。

Ansibleで操作を行う際のグループ名を指定してください。

In [ ]:
# (例)
# target_group = 'Moodle'

target_group = 

削除候補となるインスタンスのIDリストを表示します。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(target_group)
for x in gvars['instance_ids']:
    print(x)

削除対象のインスタンスIDを指定してください。

In [ ]:
# (例)
# target_instance_id = 'i-xxxxxxxxxxxxxxxxx'

target_instance_id = 

## EC2インスタンスの削除

Moodle環境を構築したEC2インスタンスを削除します。

In [ ]:
import os
import json
os.environ['PATH'] += f':{os.environ["HOME"]}/.local/bin'
instance_ids=json.dumps([target_instance_id])
!pipenv run ansible localhost -c local -m amazon.aws.ec2_instance -a \
    'instance_ids={target_instance_id} region={gvars["aws_region"]} \
    state=absent'

## Ansible 設定ファイルの後始末

Ansibleに関する設定ファイルの後始末を行います。

group_vars に記録されているパラメータを無効にするために group_vars ファイルをリネームします。

In [ ]:
!mv group_vars/{target_group}.yml  group_vars/{target_group}.yml.bak

インベントリから対象となるグループを削除します。

In [ ]:
%run scripts/group.py
remove_group_from_inventory_yml(target_group)

# 削除前との差分を表示します。
! ! diff -u inventory.yml.bak inventory.yml